# Pandas Explode Column

In [1]:
import pandas as pd

## Data

In [2]:
df = pd.DataFrame({'csv' : ['value1',
                             'value1,value2',
                             'value1,value2,value3,value4,value5,value6',
                             'value1',
                             'value1,value2,value3,value4']})

In [3]:
df.index.names = ['id']

In [4]:
df

,csv
id,
0,value1
1,"value1,value2"
2,"value1,value2,value3,value4,value5,value6"
3,value1
4,"value1,value2,value3,value4"


## Explode

### Method 1

In [5]:
data = {i:row.csv.split(',') for i,row in df.iterrows()}

In [6]:
df_exploded = pd.DataFrame.from_dict(data, orient='index').stack().to_frame()

In [7]:
df_exploded.columns = ['value']
df_exploded.index.names = ['id', 'sequence']

In [8]:
df_exploded

value
id sequence        
0  0         value1
1  0         value1
   1         value2
2  0         value1
   1         value2
   2         value3
   3         value4
   4         value5
   5         value6
3  0         value1
4  0         value1
   1         value2
   2         value3
   3         value4

### Method 2

In [9]:
df_exploded = df['csv'].str.split(pat=',', expand=True).stack().to_frame()

In [10]:
df_exploded.columns = ['value']
df_exploded.index.names = ['id', 'sequence']

In [11]:
df_exploded

value
id sequence        
0  0         value1
1  0         value1
   1         value2
2  0         value1
   1         value2
   2         value3
   3         value4
   4         value5
   5         value6
3  0         value1
4  0         value1
   1         value2
   2         value3
   3         value4

## Join

In [12]:
df_exploded = df_exploded.reset_index(level='sequence')

In [13]:
df.merge(df_exploded, how='left', left_index=True, right_index=True)

,csv,sequence,value
id,,,
0,value1,0,value1
1,"value1,value2",0,value1
1,"value1,value2",1,value2
2,"value1,value2,value3,value4,value5,value6",0,value1
2,"value1,value2,value3,value4,value5,value6",1,value2
2,"value1,value2,value3,value4,value5,value6",2,value3
2,"value1,value2,value3,value4,value5,value6",3,value4
2,"value1,value2,value3,value4,value5,value6",4,value5
2,"value1,value2,value3,value4,value5,value6",5,value6
